In [1]:
# Импорт необходимых библиотек
import pandas as pd
import re
import datetime as dt

In [2]:
# Функция отображения вспомогательной информации
def prn_info(p_df, p_col=None):
#     print('    *** value_counts of ['+p_col+']:')
#     print(p_df[p_col].value_counts())
#     print('    ***    non-null rows qnty of ['+p_col+']:',p_df[p_col].value_counts().sum())
    print(p_df.info())

In [3]:
# !!! Вместо этой функции используется to_csv() !!!

# Функция записи колонки с новой фичей в файл
# Для последующего merge используется колонка ID_TA
# def write_to_csv(p_df, p_col):
#     idxLst = p_df.index.to_list()
#     header = 'ID_TA,'+p_col+'\n'

#     ouf = open('new_cols/'+p_col+'.csv', 'w')
#     ouf.write(header)
#     for idx in idxLst:
#         s = str(p_df['ID_TA'].iloc[idx])+','+str(p_df[p_col].iloc[idx])+'\n'
#         ouf.write(s)
#     ouf.close()

## //--> ===== Заполнение пропусков =====

### ['Cuisine Style']

In [4]:
def fill_CuisineStyle(p_df_in):
    p_df_in['Cuisine Style'] = p_df_in['Cuisine Style'].fillna('noname')
    return p_df_in

### ['Number of Reviews']
Определить ['Number of Reviews']=NaN по кол-ву ['Reviews'] ('[[], []]' = 0/1/2)

In [5]:
def fill_NumberOfReviews(p_df_in, p_df_out):
    cnt=0
    idxList = p_df_in.index[p_df_in['Number of Reviews'].isna()].to_list()
    
    for idx in idxList:
        cnt += 1
#         if cnt>5:
#             break
        if p_df_in.loc[idx, 'Reviews'] == '[[], []]':
            p_df_in.loc[idx, 'Number of Reviews'] = 0
        else:
            lst = re.findall(r'\d\d\/\d\d\/\d{4}', p_df_in['Reviews'].iloc[idx])
            p_df_in.loc[idx, 'Number of Reviews'] = len(lst)

    p_df_out = pd.read_csv('../data/all_cols.csv') #инициализация
    p_df_out['Number of Reviews'] = p_df_in['Number of Reviews'] #копируем
    p_df_out.to_csv('../data/all_cols.csv', index=False) #скидываем в файл
    prn_info(p_df_out)
    return p_df_out

### ['newPriceLevel']
Среднее знач-е при фиксированных City, Rating, 'Number of Reviews', newQntyCS
# !!! Пока не используется

In [6]:
# cnt=0
# d = {}
# idxList = df.index[df['Price Range'].isna()].to_list()

# for idx in idxList:
#     cnt += 1
#     #print(idx)
#     if cnt>5:
#         break
#     #print(df['newPriceLevel'].iloc[idx])
#     city = df['City'].iloc[idx]
#     rating = df['Rating'].iloc[idx]
#     NofRev = df['Number of Reviews'].iloc[idx]
#     QntyCS = df['newQntyCS'].iloc[idx]
#     key = city+'#'+rating+'#'+str(NofRev)#+'#'+to_string(QntyCS)
    
#     if key not in d.keys():
#         mean_val = df.loc[(df['City'] == city) & (df['Rating']==rating) &
#              (df['Number of Reviews']==NofRev) & (df['newQntyCS']==QntyCS) & 
#              (df['newPriceLevel'] != -1)
#         ].newPriceLevel.mean()
#         if pd.isna(mean_val):
#             #расширяем границы - убираем условие (df['Number of Reviews']==df['Number of Reviews'].iloc[idx])
#             mean_val = df.loc[(df['City'] == df['City'].iloc[idx]) & 
#                                  (df['Rating']==df['Rating'].iloc[idx]) &
#                                  (df['newQntyCS']==df['newQntyCS'].iloc[idx]) & 
#                                  (df['newPriceLevel'] != -1)
#                             ].newPriceLevel.mean()
#             if pd.isna(mean_val):
#                 #расширяем границы - убираем условие (df['Rating']==df['Rating'].iloc[idx])
#                 mean_val = df.loc[(df['City'] == df['City'].iloc[idx]) & 
#                                      (df['newQntyCS']==df['newQntyCS'].iloc[idx]) & 
#                                      (df['newPriceLevel'] != -1)
#                                 ].newPriceLevel.mean()
        
#     d[key] = round(mean_val)
#     df['newPriceLevel'].iloc[idx] = d[key]
#     #print(idx,':',df['newPriceLevel'].iloc[idx])

# prn_info('newIsPriceLevMin')
# prn_info('newIsPriceLevMdl')
# prn_info('newIsPriceLevMax')

## ===== <--//

## ===== Генерация новых признаков

## //--> *** Цены

### Признаки попадания уровня цен в ресторане в определенный диапазон:
- ['newIsPriceLevMin'] - min (Да/Нет)
- ['newIsPriceLevMdl'] - middle (Да/Нет)
- ['newIsPriceLevMax'] - max (Да/Нет)

In [7]:
def create_IsPriceLev(p_df_in, p_df_out):
    p_df_in['newIsPriceLevMin'] = 0
    p_df_in['newIsPriceLevMdl'] = 0
    p_df_in['newIsPriceLevMax'] = 0

    #пропущенные знач-я определены как мин. уровень цен
    p_df_in.loc[p_df_in['Price Range'].isna(), ['newIsPriceLevMin']] = 1
    
    p_df_in.loc[p_df_in['Price Range'] == '$', ['newIsPriceLevMin']] = 1
    p_df_in.loc[p_df_in['Price Range'] == '$$ - $$$', ['newIsPriceLevMdl']] = 1
    p_df_in.loc[p_df_in['Price Range'] == '$$$$', ['newIsPriceLevMax']] = 1

    p_df_out = pd.read_csv('../data/all_cols.csv') #инициализация
    p_df_out['newIsPriceLevMin'] = p_df_in['newIsPriceLevMin'] #копируем
    p_df_out['newIsPriceLevMdl'] = p_df_in['newIsPriceLevMdl'] #копируем
    p_df_out['newIsPriceLevMax'] = p_df_in['newIsPriceLevMax'] #копируем
    p_df_out.to_csv('../data/all_cols.csv', index=False) #скидываем в файл
    prn_info(p_df_out)
    return p_df_out

## Цены *** <--//

## //--> *** Города

### Новый признак ['newIsCapital'] - является ли город нахождения ресторана столицей (1=Да / 0=Нет)

In [8]:
def create_IsCapital(p_df_in, p_df_out):
    p_df_in['newIsCapital'] = 1 #сначала записываем везде 1
    #теперь определяем НЕстолицы:
    p_df_in.loc[(p_df_in['City']=='Munich') | (p_df_in['City']=='Oporto') | 
                (p_df_in['City']=='Milan') | (p_df_in['City']=='Barcelona') | 
                (p_df_in['City']=='Zurich') | (p_df_in['City']=='Lyon') | 
                (p_df_in['City']=='Hamburg') | (p_df_in['City']=='Geneva') | 
                (p_df_in['City']=='Krakow'), 'newIsCapital'] = 0

    p_df_out = pd.read_csv('../data/all_cols.csv') #инициализация
    p_df_out['newIsCapital'] = p_df_in['newIsCapital'] #копируем
    p_df_out.to_csv('../data/all_cols.csv', index=False) #скидываем в файл
    prn_info(p_df_out)
    return p_df_out

### Новые признаки ['newIsRankRangeQ1-Q4'] - диапазон попадания Ranking для каждого города

In [9]:
def create_IsRankRange(p_df_in, p_df_out):
    p_df_in['newIsRankRangeQ1'] = 0
    p_df_in['newIsRankRangeQ2'] = 0
    p_df_in['newIsRankRangeQ3'] = 0
    p_df_in['newIsRankRangeQ4'] = 0

    for city in p_df_in.City.unique():
        rr = p_df_in.loc[p_df_in.City==city].Ranking.value_counts(bins=4)
        rr = pd.DataFrame(rr)

        lst_rr = []
        for i in range(4):
            lst = []
            intrv = str(rr.index[i])
            for s in intrv.strip('( )[ ]').split(', '):
                lst.append(float(s))
            lst_rr += [lst]
        lst_rr.sort()

        idxList = p_df_in.index[p_df_in.City == city].to_list()
        for idx in idxList:
            r = p_df_in.loc[idx, 'Ranking']
            if lst_rr[0][0] < r <= lst_rr[0][1]:
                p_df_in.loc[idx, 'newIsRankRangeQ1'] = 1
            elif lst_rr[1][0] < r <= lst_rr[1][1]:
                p_df_in.loc[idx, 'newIsRankRangeQ2'] = 1
            elif lst_rr[2][0] < r <= lst_rr[2][1]:
                p_df_in.loc[idx, 'newIsRankRangeQ3'] = 1
            elif lst_rr[3][0] < r <= lst_rr[3][1]:
                p_df_in.loc[idx, 'newIsRankRangeQ4'] = 1
        #break

    p_df_out = pd.read_csv('../data/all_cols.csv') #инициализация
    p_df_out['newIsRankRangeQ1'] = p_df_in['newIsRankRangeQ1']
    p_df_out['newIsRankRangeQ2'] = p_df_in['newIsRankRangeQ2']
    p_df_out['newIsRankRangeQ3'] = p_df_in['newIsRankRangeQ3']
    p_df_out['newIsRankRangeQ4'] = p_df_in['newIsRankRangeQ4']
    p_df_out.to_csv('../data/all_cols.csv', index=False) #скидываем в файл
    prn_info(p_df_out)
    return p_df_out

## *** <--//

## //--> *** Кухни

### Новый признак ['newQntyCS'] - кол-во представленных в ресторане кухонь
# ---
### Новые признаки ['newQntyCSeql01'], ['newQntyCS02-05'], ['newQntyCSgrt05']:
 - для каждого ресторана кол-во представленных в нем кухонь разбить по колонкам "Кол-во кухонь = 1", "Кол-во кухонь от 2 до 5", "Кол-во кухонь более 5"

In [10]:
def create_CS_info(p_df_in, p_df_out):
    p_df_in['newQntyCS'] = 0
    p_df_in['newIsQntyCSeql01'] = 0
    p_df_in['newIsQntyCS02-05'] = 0
    p_df_in['newIsQntyCSgrt05'] = 0
    idx = -1
    cuisines = set()  # создаём пустое множество для хранения уникальных значений кухонь

    for v in p_df_in['Cuisine Style'].apply(lambda x: str(x).strip('[]')):
        idx += 1
#         if idx>5:
#             break
        QntyCS = 0

        if pd.isna(str(v)) or str(v).lower() == 'nan':
            QntyCS = 1
            p_df_in.loc[idx, 'newQntyCS'] = 1
        elif ',' not in v:
            QntyCS = 1
            p_df_in.loc[idx, 'newQntyCS'] = 1
            cuisines.add(v.strip())
        else:
            for cs in v.split(','):
                QntyCS += 1
                p_df_in.loc[idx, 'newQntyCS'] += 1
                cuisines.add(cs.strip())

        if QntyCS == 1:
            p_df_in.loc[idx, 'newIsQntyCSeql01'] = 1
        if 2 <= QntyCS <= 5:
            p_df_in.loc[idx, 'newIsQntyCS02-05'] = 1
        elif 5 < QntyCS:
            p_df_in.loc[idx, 'newIsQntyCSgrt05'] = 1

    p_df_out = pd.read_csv('../data/all_cols.csv') #инициализация
    p_df_out['newQntyCS'] = p_df_in['newQntyCS'] #копируем
    p_df_out['newIsQntyCSeql01'] = p_df_in['newIsQntyCSeql01'] #копируем
    p_df_out['newIsQntyCS02-05'] = p_df_in['newIsQntyCS02-05'] #копируем
    p_df_out['newIsQntyCSgrt05'] = p_df_in['newIsQntyCSgrt05'] #копируем
    p_df_out.to_csv('../data/all_cols.csv', index=False) #скидываем в файл
    prn_info(p_df_out)
    
    cs = pd.DataFrame()
    idx = -1
    for s in cuisines:
        #print(s)
        idx += 1
        cs.loc[idx, 'cuisine'] = s.strip("'")
    cs.to_csv('../data/cuisines.csv', index=False) #скидываем в файл
        
    return p_df_out

### Новые признаки: наименование колонки по названию стиля кухни
Колонки заполняются значениями: 1 - в ресторане есть эта кухня / 0 - нет такой кухни

In [11]:
def create_CS_cols(p_df_in, p_df_out):
    p_df_out = pd.read_csv('../data/all_cols.csv') #инициализация
    p_df_out['Cuisine Style'] = p_df_in['Cuisine Style']
    p_df_out['Cuisine Style'] = p_df_out['Cuisine Style'].fillna('noname')

    cuisines = pd.read_csv('../data/cuisines.csv')
    for item in cuisines.cuisine:
        p_df_out[item] = p_df_out['Cuisine Style'].apply(lambda x: 1 if item in x else 0)

    p_df_out = p_df_out.drop(['Cuisine Style'], axis=1)
    p_df_out.to_csv('../data/all_cols.csv', index=False) #скидываем в файл
    prn_info(p_df_out)
    return p_df_out

### Новый признак ['newIsUniqueCS'] - присутствует ли в ресторане тип кухни в единственном экземпляре среди всех ресторанов (1=Да / 0=Нет)
# !!! Пока не используется

In [12]:
# df['newIsUniqueCS'] = 0 #для нового признака
# #Ручное заполнение

# #Список кухонь, встречающихся по 1 разу:
# ['Mexican', 'Latin', 'Salvadoran', 'Central American', 'Spanish']
# ['Asian', 'Thai', 'Yunnan', 'Vegetarian Friendly', 'Vegan Options']
# ['Chinese', 'Xinjiang']
# ['Chinese', 'Asian', 'Thai', 'Burmese']
# ['Mexican', 'American', 'European', 'Latvian', 'Eastern European', 'Hungarian']

# idxList = df.index[(df['Cuisine Style'] == 
#    "['Mexican', 'Latin', 'Salvadoran', 'Central American', 'Spanish']") |
#    (df['Cuisine Style'] == 
#     "['Asian', 'Thai', 'Yunnan', 'Vegetarian Friendly', 'Vegan Options']") |
#    (df['Cuisine Style'] == "['Chinese', 'Xinjiang']") |
#    (df['Cuisine Style'] == "['Chinese', 'Asian', 'Thai', 'Burmese']") |
#    (df['Cuisine Style'] == 
#     "['Mexican', 'American', 'European', 'Latvian', 'Eastern European', 'Hungarian']")
#       ].to_list()
# df.loc[idxList, 'newIsUniqueCS'] = 1

# write_to_csv('newIsUniqueCS')

# prn_info('newIsUniqueCS')

## *** <--//

## //--> *** Отзывы

### Новый признак ['newQntyLastReview'] - кол-во свежих отзывов (0=Нет отзывов / 1 / 2)
# !!! Пока не используется
# ---
### Новые признаки ['newLastRevYear'], ['newLastRevMonth'], ['newLastRevDofM'], ['newLastRevDofW']
 - самый свежий отзыв: год, месяц, день месяца, день недели, час суток, когда оставлен отзыв
# !!! Пока не используется

# ---
### Новый признак ['newIsPositiveRev']
 - newIsPositiveRev - явно позитивный отзыв (1=Да / 0=Нет)

In [13]:
def create_IsPositiveRev(p_df_in, p_df_out):
    # df['newQntyLastReview'] = 0
    # df['newLastRevYear'] = 1900
    # df['newLastRevMonth'] = 1
    # df['newLastRevDofM'] = 1
    # df['newLastRevDofW'] = 1
#     df['newIsPositiveRev'] = 0

#     cnt = 0
#     idxList = df.index[df['Reviews'] != '[[], []]'].to_list()
#     for idx in idxList:
#         cnt += 1
    #     if cnt>5:
    #         break

        #newQntyLastReview
    #     lst = re.findall(r'\d\d\/\d\d\/\d{4}', df['Reviews'].iloc[idx])
    #     df.loc[idx, 'newQntyLastReview'] = len(lst)

        #newLastRevYear, newLastRevMonth, newLastRevDofM, newLastRevDofW
    #     t = None
    #     if len(lst) == 1:
    #         t = pd.to_datetime(lst[0])
    #     elif len(lst) == 2:
    #         t1 = pd.to_datetime(lst[0])
    #         t2 = pd.to_datetime(lst[1])
    #         if t1>t2:
    #             t = t1
    #         else:
    #             t = t2
    #     if not pd.isnull(t):
    #         #print(t, ',', t.hour)
    #         df.loc[idx, 'newLastRevYear'] = t.year
    #         df.loc[idx, 'newLastRevMonth'] = t.month
    #         df.loc[idx, 'newLastRevDofM'] = t.day
    #         df.loc[idx, 'newLastRevDofW'] = t.weekday()

        #newIsPositiveRev
#         lst = re.findall(r'good|delic|lovin|wonderf|awesom|best|great|fantast|pleasan', 
#                          str(df['Reviews'].iloc[idx]).lower())
#         if len(lst) > 0:
#              df.loc[idx, 'newIsPositiveRev'] = 1

    p_df_in['newIsPositiveRev'] = p_df_in['Reviews'].apply(
        lambda x: 1 if len(re.findall(r'good|delic|lovin|wonderf|awesom|best|great|fantast|pleasan', 
                         str(x.lower()))) > 0 else 0)
                
    p_df_out = pd.read_csv('../data/all_cols.csv') #инициализация
    p_df_out['newIsPositiveRev'] = p_df_in['newIsPositiveRev']
    p_df_out.to_csv('../data/all_cols.csv', index=False) #скидываем в файл
    prn_info(p_df_out)
    return p_df_out

### Новый признак ['newQntyDaysBetwRev'] - кол-во дней между двумя последними отзывами (365x200=73000 - нет ни одного отзыва /  365x100=36500 - есть только 1 отзыв / N - кол-во дней между двумя последними отзывами)

# Пока не придумал, как лучше заполнить - пока пропуск

In [14]:
# df['newQntyDaysBetwRev'] = 365 * 200
# cnt = 0
    
# idxList = df.index[df['Reviews'] != '[[], []]'].to_list()
# for idx in idxList:
#     cnt += 1
# #     if cnt>5:
# #         break
#     Tdiff = dt.timedelta(days=0)
#     lst = re.findall(r'\d\d\/\d\d\/\d{4}', df['Reviews'].iloc[idx])
#     if len(lst) == 1:
#         df.loc[idx, 'newQntyDaysBetwRev'] = 365 * 100
#     else:
#         t1 = pd.to_datetime(lst[0])
#         t2 = pd.to_datetime(lst[1])
#         if t1 > t2:
#             Tdiff = t1 - t2
#         else:
#             Tdiff = t2 - t1
#         df.loc[idx, 'newQntyDaysBetwRev'] = Tdiff.days
#         #print(Tdiff, Tdiff.days)

# write_to_csv('newQntyDaysBetwRev')

# prn_info('Reviews')
# prn_info('newQntyDaysBetwRev')

## *** <--//

## //--> *** Рестораны

### Новый признак ['newIsChain'] - сетевой ресторан (1=Да / 0=Нет)

In [15]:
def create_IsChain(p_df_in, p_df_out):
    p_df_in['newIsChain'] = 1
    df_chain = pd.DataFrame(p_df_in.groupby(p_df_in['Restaurant_id']).newIsChain.count())
    df_chain.columns = ['cnt']
    idxList = df_chain.index[df_chain['cnt'] == 1].to_list() #это НЕсетевые рестораны

    p_df_in['newIsChain'] = p_df_in['Restaurant_id'].apply(lambda x: 0 if x in idxList else 1)

    p_df_out = pd.read_csv('../data/all_cols.csv') #инициализация
    p_df_out['newIsChain'] = p_df_in['newIsChain']
    p_df_out.to_csv('../data/all_cols.csv', index=False) #скидываем в файл
    prn_info(p_df_out)
    return p_df_out

### Новый признак ['RankingInverse'] - обратная величина Ranking

In [16]:
def create_RankingInverse(p_df_in, p_df_out):
    p_df_out = pd.read_csv('../data/all_cols.csv') #инициализация
    p_df_out['RankingInverse'] = 1 / p_df_out['Ranking']
    p_df_out.to_csv('../data/all_cols.csv', index=False) #скидываем в файл
    prn_info(p_df_out)
    return p_df_out    

## *** <--//

In [17]:
%%time
##### Тестирование
# df = pd.read_csv('../data/main_task.csv')
# X_all_cols = pd.read_csv('../data/all_cols.csv')

# X_all_cols = create_IsChain(df, X_all_cols)
# X_all_cols.head()

Wall time: 0 ns


In [18]:
# X_all_cols.newIsChain.value_counts()

In [19]:
# p_df_in = pd.read_csv('../data/main_task.csv')

In [20]:
# for city in p_df_in.City.unique():
#     print(city)
#     rr = p_df_in.loc[p_df_in.City==city].Ranking.value_counts(bins=4)
#     rr = pd.DataFrame(rr)

#     lst_rr = []
#     for i in range(4):
#         lst = []
#         intrv = str(rank_range.index[i])
#         for s in intrv.strip('( )[ ]').split(', '):
#             lst.append(float(s))
#         lst_rr += [lst]
#     lst_rr.sort()

#     break

# lst_rr